# Classificazione di Aspect e Opinion con ModernBERT su Laptop-ACOS

In [ ]:
# Import delle librerie necessarie
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os
import sys
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
import torch.nn as nn
import bitsandbytes as bnb

from evaluate import load
from tqdm import tqdm

from torch.amp import autocast, GradScaler # Per Mixed Precision

print("Librerie caricate.")

Librerie caricate.


### Impostazioni per la riproducibilità 

In [3]:
def set_seed(seed_value=42):
    """Imposto i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposto anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


In [4]:
WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters
config = {
    "learning_rate": 5e-5,
    "epochs": 5,
    "batch_size": 16,
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Laptop-ACOS",
    "seed": 42,
    'patience': 2  # Per Early Stopping
}

# 2. Inizializzazione del Run
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config,
    name=f"run_{config['model_name']}_{config['dataset']}"
)

print(f"W&B inizializzato per il progetto: {wandb.run.project}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B inizializzato per il progetto: BigData-TextMining-ACOS


## PyTorch Dataset & DataLoader Construction

### Creazione di PyTorch Dataset e DataLoader
In questa fase, trasformiamo i nostri DataFrame Pandas (strutture dati tabellari) in oggetti Dataset e DataLoader di PyTorch. Questo passaggio è il "ponte" necessario per alimentare il modello ModernBERT durante l'addestramento.

#### Obiettivi di questa sezione:

  1. Standardizzazione dei Dati (ACOSDataset):

       * I modelli basati su Transformer non possono leggere direttamente i DataFrame. La classe ACOSDataset estrae le liste di input_ids, attention_mask e labels e le converte in Tensori PyTorch (torch.tensor).

       * Viene utilizzato il tipo di dato torch.long, richiesto dai layer di embedding e dalle funzioni di calcolo della Loss per task di classificazione.

  2. Gestione del Caricamento (DataLoader):

      * Batching: Invece di caricare l'intero dataset in memoria (rischioso per la GPU), i dati vengono divisi in piccoli blocchi chiamati Batch (nel nostro caso di dimensione 16).

      * Shuffling (Solo Training): Utilizziamo shuffle=True nel train_loader per rimescolare l'ordine delle frasi a ogni epoca. Questo impedisce al modello di imparare l'ordine sequenziale dei dati, costringendolo invece a focalizzarsi sui pattern linguistici reali.

      * Efficienza: I DataLoader gestiscono il caricamento dei dati in parallelo, ottimizzando i tempi di addestramento sulla GPU.

In [6]:

# --- 1. CARICAMENTO DEI DATASET SALVATI  ---
cartella_dati = "data_allineati"

print("📂 Caricamento dei dataset pre-processati...")
# Carichiamo i Laptop
df_train_align_laptop = pd.read_pickle(os.path.join(cartella_dati, "train_laptop_aligned.pkl"))
df_dev_align_laptop = pd.read_pickle(os.path.join(cartella_dati, "dev_laptop_aligned.pkl"))
df_test_align_laptop = pd.read_pickle(os.path.join(cartella_dati, "test_laptop_aligned.pkl"))

class ACOSDataset(Dataset):
    def __init__(self, df):
        # Estraiamo le colonne che abbiamo generato nella fase di allineamento
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['labels'].tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Convertiamo le liste in Tensori di PyTorch (LongTensor per ID e Label)
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# --- CREAZIONE DELLE ISTANZE ---

# Creiamo i dataset per il dominio Laptop
train_dataset_laptop = ACOSDataset(df_train_align_laptop)
dev_dataset_laptop = ACOSDataset(df_dev_align_laptop)
test_dataset_laptop = ACOSDataset(df_test_align_laptop)


# --- CONFIGURAZIONE DATALOADERS ---

BATCH_SIZE = 16 # Numero di frasi analizzate contemporaneamente

train_loader_laptop = DataLoader(train_dataset_laptop, batch_size=BATCH_SIZE, shuffle=True)
dev_loader_laptop = DataLoader(dev_dataset_laptop, batch_size=BATCH_SIZE)
test_loader_laptop = DataLoader(test_dataset_laptop, batch_size=BATCH_SIZE)



print(f"Dataset e DataLoaders creati con successo!")
print(f"Esempi nel set di Training LAPTOP: {len(train_dataset_laptop)}")


📂 Caricamento dei dataset pre-processati...
Dataset e DataLoaders creati con successo!
Esempi nel set di Training LAPTOP: 2934


### Definizione e l'Inizializzazione del Modello di Token Classification.

1. Caricare il "Cervello" (ModernBERT Pre-trained)
Dobbiamo scaricare il modello ModernBERT-base dal repository di Hugging Face. In questa fase, il modello sa già "leggere" e "capire" la lingua inglese perché è stato addestrato su miliardi di testi, ma non sa ancora nulla del tuo task specifico (ACOS). È come un laureato in lingue che però non ha mai lavorato in un ristorante o in un negozio di computer.

2. Aggiungere la "Testa" di Classificazione
ModernBERT normalmente restituisce dei vettori numerici (embedding) per ogni parola. Noi dobbiamo aggiungere sopra questi vettori uno strato finale chiamato Linear Layer (o testa di classificazione).

   * Questo strato prenderà l'output di ModernBERT e lo "schiaccerà" su 5 classi possibili: 0 (O), 1 (B-ASP), 2 (I-ASP), 3 (B-OPI), 4 (I-OPI).

   * Il modello dovrà imparare a mappare ogni pezzetto di frase a una di queste cinque etichette.

3. Configurare la Strategia di Apprendimento (Optimizer & Loss)
Dobbiamo dare al modello gli strumenti per imparare dai suoi errori:

  * Loss Function (Funzione di Perdita): Useremo la CrossEntropyLoss. È il "voto" che diamo al modello. Se il modello dice che "pizza" è un'opinione (B-OPI) ma il tuo dataset dice che è un aspetto (B-ASP), la Loss sarà alta. Il modello cercherà di abbassarla il più possibile.

   * Optimizer (Ottimizzatore): Di solito si usa AdamW. È l'algoritmo che decide "come" e "quanto" cambiare i pesi interni del modello per correggere gli errori.

   * Learning Rate: La velocità con cui il modello impara. Se è troppo alta, il modello è "frettoloso" e sbaglia; se è troppo bassa, non imparerà mai.

In [ ]:

# --- 1. CONFIGURAZIONE DEL DEVICE ---
# Se hai una GPU NVIDIA, userà 'cuda'. Se hai un Mac M1/M2, userà 'mps'. Altrimenti 'cpu'.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f" GPU Trovata: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(" Acceleratore Apple Metal (MPS) Trovato")
else:
    device = torch.device("cpu")
    print(" Nessuna GPU trovata. L'addestramento sarà lento.")

# --- 2. CARICAMENTO DI MODERNBERT (IL "CERVELLO") + TESTA DI CLASSIFICAZIONE ---
# Definiamo le 5 etichette: 0=O, 1=B-ASP, 2=I-ASP, 3=B-OPI, 4=I-OPI
NUM_LABELS = 5 

print("Scaricamento e configurazione di ModernBERT...")
model = AutoModelForTokenClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=NUM_LABELS
)

# Spostiamo il modello sul dispositivo di calcolo (GPU/MPS/CPU)
model.to(device)

# --- 3. CONFIGURAZIONE DELL'OTTIMIZZATORE E DELLA LOSS ---

# A. Optimizer (AdamW 8-bit)
# Usiamo il Learning Rate standard di 5e-5 come definito nei parametri sperimentali 
# e la versione a 8-bit per non saturare la memoria
optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=5e-5)

# B. Loss Function (CrossEntropyLoss)
# La funzione che calcola l'errore tra la predizione del modello e le label reali.
# Nota: 'ignore_index=-100' è lo standard di PyTorch per ignorare i token di padding nel calcolo dell'errore.
loss_fn = nn.CrossEntropyLoss()

print("\n" + "="*50)
print("MODELLO PRONTO PER IL TRAINING")
print("="*50)
print(f"Architettura: ModernBERT-base")
print(f"Task: Token Classification (Estrazione Aspetti & Opinioni)")
print(f"Numero di Classi: {NUM_LABELS}")
print(f"Optimizer: AdamW (lr=5e-5)")
print(f"Loss Function: CrossEntropyLoss")

 Acceleratore Apple Metal (MPS) Trovato
Scaricamento e configurazione di ModernBERT...


Loading weights:   0%|          | 0/136 [00:00<?, ?it/s]

ModernBertForTokenClassification LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     | 
------------------+------------+-
decoder.bias      | UNEXPECTED | 
classifier.weight | MISSING    | 
classifier.bias   | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.



MODELLO PRONTO PER IL TRAINING
Architettura: ModernBERT-base
Task: Token Classification (Estrazione Aspetti & Opinioni)
Numero di Classi: 5
Optimizer: AdamW (lr=5e-5)
Loss Function: CrossEntropyLoss


In [ ]:
# --- 1. CONFIGURAZIONE AVANZATA MEMORIA ---
# Gradient Checkpointing: risparmia tantissima VRAM ricalcolando i passaggi intermedi
model.gradient_checkpointing_enable()

# Parametri per simulare un batch size maggiore
accumulation_steps = config.get('accumulation_steps', 4) 
patience = config.get('patience', 2)
patience_counter = 0

optimizer = bnb.optim.AdamW8bit(
    model.parameters(), 
    lr=config['learning_rate']
)
# Scaler per Mixed Precision (fondamentale per evitare l'OOM)
scaler = GradScaler() 

total_steps = (len(train_loader_laptop) // accumulation_steps) * config['epochs']
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# --- 2. FUNZIONI DI SUPPORTO OTTIMIZZATE ---

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Usiamo autocast anche in valutazione
            with autocast(device_type='cuda'):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            
            total_loss += outputs.loss.item()
    return total_loss / len(data_loader)

def train_epoch(model, data_loader, optimizer, scheduler, device, epoch_idx, scaler, accumulation_steps):
    model.train()
    total_loss = 0
    optimizer.zero_grad() # Reset iniziale
    
    loop = tqdm(data_loader, leave=True)
    
    for i, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # A. Mixed Precision Forward Pass
        with autocast(device_type='cuda'):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / accumulation_steps 
        
        # B. Backward Pass con Scaler
        scaler.scale(loss).backward()
        
        # C. Update Pesi ogni 'accumulation_steps'
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item() * accumulation_steps
        wandb.log({"batch_loss": loss.item() * accumulation_steps})
        loop.set_description(f"Epoca {epoch_idx + 1}")
        loop.set_postfix(loss=loss.item() * accumulation_steps)

    return total_loss / len(data_loader)

# --- 3. CICLO DI ADDESTRAMENTO ---

print(f"🚀 Training su LAPTOP: {config['epochs']} epoche | Device: {device}")
print(f"📦 Accumulo Gradienti ogni {accumulation_steps} step | FP16 Attivato")

best_valid_loss_laptop = float('inf')


for epoch in range(config['epochs']):
    print(f"\n--- Epoca {epoch+1}/{config['epochs']} ---")
    
    # 1. Training
    train_loss_laptop = train_epoch(model, train_loader_laptop, optimizer, scheduler, device, epoch, scaler, accumulation_steps)
    
    # 2. Validazione
    valid_loss_laptop = evaluate_model(model, dev_loader_laptop, device)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print(f"📉 Train Loss: {train_loss_laptop:.4f} | 🔍 Valid Loss: {valid_loss_laptop:.4f}")
    
    # 3. Log metriche epoca su W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": train_loss_laptop,
        "valid_loss_epoch": valid_loss_laptop
    })
    
    # --- LOGICA EARLY STOPPING & CHECKPOINT ---
    
    # Se il modello migliora (la valid loss scende)
    if valid_loss_laptop < best_valid_loss_laptop:
        best_valid_loss_laptop = valid_loss_laptop
        patience_counter = 0  # ### NUOVO: Resettiamo la pazienza ###
        
        print(f"💾 Miglior modello trovato (Loss: {best_valid_loss_laptop:.4f})! Salvataggio...")
        
        import os
        # Creiamo la cartella se non esiste (sicurezza aggiuntiva)
        if not os.path.exists("./best_model_laptop"):
            os.makedirs("./best_model_laptop")
            
        model.save_pretrained("./best_model_laptop")
        
    # Se il modello NON migliora
    else:
        patience_counter += 1  # ### NUOVO: Incrementiamo il contatore ###
        print(f"⚠️ Nessun miglioramento. Patience: {patience_counter}/{patience}")
        
        # Se abbiamo esaurito la pazienza
        if patience_counter >= patience:
            print(f"\n🛑 EARLY STOPPING ATTIVATO! Interruzione all'epoca {epoch+1}.")
            break # Esce dal ciclo for

print("\n✅ Fine Addestramento.")
wandb.finish()

🚀 Inizio Training su LAPTOP: 5 epoche su mps
🛑 Early Stopping configurato con Patience = 2

--- Epoca 1/5 ---


  0%|          | 0/184 [00:00<?, ?it/s]

Epoca 1: 100%|██████████| 184/184 [1:01:52<00:00, 20.17s/it, loss=0.0488]


📉 Train Loss: 0.0710 | 🔍 Valid Loss: 0.0343
💾 Miglior modello trovato (Loss: 0.0343)! Salvataggio...


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoca 4: 100%|██████████| 184/184 [1:08:28<00:00, 22.33s/it, loss=0.00218]


📉 Train Loss: 0.0098 | 🔍 Valid Loss: 0.0330
⚠️ Nessun miglioramento. Patience: 1/2

--- Epoca 5/5 ---


Epoca 5: 100%|██████████| 184/184 [1:19:17<00:00, 25.86s/it, loss=0.0117] 


📉 Train Loss: 0.0048 | 🔍 Valid Loss: 0.0340
⚠️ Nessun miglioramento. Patience: 2/2

🛑 EARLY STOPPING ATTIVATO! Interruzione all'epoca 5.

✅ Fine Addestramento.


batch_loss,█▅▅▄▂▃▂▃▂▃▃▂▂▃▃▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▃▁▁▁▁▁▁
epoch,▁▃▅▆█
train_loss_epoch,█▃▂▂▁
valid_loss_epoch,█▄▁▆█
batch_loss,0.01166
epoch,5
train_loss_epoch,0.00477
valid_loss_epoch,0.03401


In [10]:

import torch
import numpy as np
from evaluate import load
from tqdm import tqdm
from transformers import AutoModelForTokenClassification

# --- A. CARICAMENTO DEL "CAMPIONE" ---
# Carichiamo i pesi migliori salvati durante il training (Epoca 3)
print("📂 Caricamento del modello migliore...")
model_path = "./best_model_laptop"
model = AutoModelForTokenClassification.from_pretrained(model_path)
model.to(device)
model.eval() # Modalità esame (spegne dropout)

# --- B. PREPARAZIONE METRICHE ---
# Carichiamo la metrica seqeval (standard per NER/ABSA)
metric = load("seqeval")

# Mappa per decodificare i numeri in etichette
# (Assicurati che corrisponda al tuo training!)
id2label = {0: 'O', 1: 'B-ASP', 2: 'I-ASP', 3: 'B-OPI', 4: 'I-OPI'}
label_list = list(id2label.values())

print("🚀 Inizio Test sul Dataset Laptop...")

# --- C. CICLO DI PREVISIONE ---
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader_laptop, desc="Test LAPTOP"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 1. Il modello predice
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # 2. Prendiamo la classe con probabilità più alta (argmax)
        preds = torch.argmax(logits, dim=-1)

        # 3. Convertiamo i numeri in etichette (pulendo i -100)
        # Dobbiamo ignorare i token speciali (-100) usati per il padding/subwords
        for i in range(len(labels)):
            true_label_row = []
            pred_label_row = []
            
            for j in range(len(labels[i])):
                if labels[i][j] != -100: # Ignoriamo i token di padding/speciali
                    true_label_row.append(id2label[labels[i][j].item()])
                    pred_label_row.append(id2label[preds[i][j].item()])
            
            true_labels.append(true_label_row)
            predictions.append(pred_label_row)

# --- D. CALCOLO E STAMPA RISULTATI ---
results = metric.compute(predictions=predictions, references=true_labels)

print("\n" + "="*50)
print("📊 RISULTATI FINALI (TEST SET - LAPTOP)")
print("="*50)

# Stampiamo le metriche generali
print(f"Overall Precision: {results['overall_precision']:.4f}")
print(f"Overall Recall:    {results['overall_recall']:.4f}")
print(f"Overall F1-Score:  {results['overall_f1']:.4f}")
print(f"Overall Accuracy:  {results['overall_accuracy']:.4f}")

print("\n🔍 Dettaglio per Classe (Quello che conta per il paper):")
print("-" * 50)
# Estraiamo le metriche specifiche per ASP e OPI
for key in results.keys():
    if key in ['ASP', 'OPI']: # Filtriamo solo le nostre classi di interesse
        print(f"🔹 {key}:")
        print(f"   Precision: {results[key]['precision']:.4f}")
        print(f"   Recall:    {results[key]['recall']:.4f}")
        print(f"   F1-Score:  {results[key]['f1']:.4f}")
        print(f"   Support:   {results[key]['number']}") # Quanti ce n'erano davvero
print("="*50)

📂 Caricamento del modello migliore...


Loading weights:   0%|          | 0/138 [00:00<?, ?it/s]

🚀 Inizio Test sul Dataset Laptop...


Test LAPTOP: 100%|██████████| 51/51 [11:25<00:00, 13.44s/it]



📊 RISULTATI FINALI (TEST SET - LAPTOP)
Overall Precision: 0.7014
Overall Recall:    0.7850
Overall F1-Score:  0.7409
Overall Accuracy:  0.9892

🔍 Dettaglio per Classe (Quello che conta per il paper):
--------------------------------------------------
🔹 ASP:
   Precision: 0.6386
   Recall:    0.7269
   F1-Score:  0.6799
   Support:   802
🔹 OPI:
   Precision: 0.7688
   Recall:    0.8452
   F1-Score:  0.8052
   Support:   775
